In [1]:
import os

In [2]:
%pwd

'c:\\Users\\saras\\OneDrive\\Desktop\\IK_mlops_assignments\\extras\\MLflow_E2E_github\\MLflow_E2E\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\saras\\OneDrive\\Desktop\\IK_mlops_assignments\\extras\\MLflow_E2E_github\\MLflow_E2E'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
import sys
print(sys.executable)

c:\Users\saras\OneDrive\Desktop\IK_mlops_assignments\extras\MLflow_E2E_github\MLflow_E2E\env\python.exe


In [7]:
!pip install PyYAML
!pip install joblib
!pip install ensure

In [8]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories


In [9]:
import mlProject
print(mlProject.__file__)

c:\users\saras\onedrive\desktop\ik_mlops_assignments\extras\mlflow_e2e_github\mlflow_e2e\src\mlProject\__init__.py


In [10]:
from mlProject.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [11]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-08 14:02:01,921: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-08 14:02:01,921: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-08 14:02:01,934: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-08 14:02:01,936: INFO: common: created directory at: artifacts]
[2024-11-08 14:02:01,938: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-08 14:02:02,740: INFO: 2280763589: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 100356
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c7eb16441bd00dc318bbc922b362147e8ed3565dc5d64a9c26ca21170ebbd49e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 18D1:18C2E0:211C04:231F5C:672E602B
Accept-Ranges: bytes
Date: Fri, 0